In [ ]:
#default_exp model.CCS

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import torch
import pandas as pd
import numpy as np

from tqdm import tqdm

from alphadeep.model.featurize import \
    parse_aa_indices, \
    get_batch_mod_feature

from alphadeep._settings import model_const

import alphadeep.model.base as model_base

from alphadeep.model.RT import evaluate_linear_regression, evaluate_linear_regression_plot


mod_feature_size = len(model_const['mod_elements'])
charge_factor = model_const['charge_factor']

In [ ]:
#export
class EncDecModelCCS(torch.nn.Module):
    def __init__(self, 
        mod_feature_size,
        dropout=0.2
    ):
        super().__init__()
        self.aa_embedding_size = 27
        
        self.dropout = torch.nn.Dropout(dropout)
        
        hidden = 256
        self.encoder = model_base.SeqEncoder(
            self.aa_embedding_size+mod_feature_size+1, 
            hidden,
            dropout=0,
            rnn_layer=2
        )

        self.decoder = model_base.LinearDecoder(
            hidden+1,
            1
        )

    def forward(self, 
        aa_indices, 
        mod_x,
        charges,
    ):
        aa_x = torch.nn.functional.one_hot(
            aa_indices, self.aa_embedding_size
        ).float()

        x = torch.cat((
            aa_x, mod_x, charges.unsqueeze(1).repeat(1, aa_x.size(1), 1)
        ), 2)
        x = self.encoder(x)
        x = self.dropout(x)
        x = torch.cat((x, charges),1)

        return self.decoder(x).squeeze(1)

In [ ]:
#export

class AlphaCCSModel(model_base.ModelImplBase):
    def __init__(self, dropout=0.2, lr=0.001):
        super().__init__()
        self.build(
            EncDecModelCCS, lr=lr, 
            dropout=dropout, 
            mod_feature_size=mod_feature_size
        )
        self.loss_func = torch.nn.L1Loss()
        self.charge_factor = charge_factor
    
    def train(self, 
        precursor_df: pd.DataFrame,
        epoch=10,
        batch_size=1024,
        verbose=False,
        verbose_each_epoch=False,
    ):
        self.model.train()

        for epoch in range(epoch):
            batch_cost = []
            _grouped = list(precursor_df.sample(frac=1).groupby('nAA'))
            rnd_nAA = np.random.permutation(len(_grouped))
            if verbose_each_epoch:
                batch_tqdm = tqdm(rnd_nAA)
            else:
                batch_tqdm = rnd_nAA
            for i_group in batch_tqdm:
                nAA, df_group = _grouped[i_group]
                df_group = df_group.reset_index(drop=True)
                for i in range(0, len(df_group), batch_size):
                    batch_end = i+batch_size-1 # DataFrame.loc[start:end] inlcudes the end

                    aa_indices = torch.LongTensor(
                        parse_aa_indices(
                            df_group.loc[i:batch_end, 'sequence'].values.astype('U')
                        )
                    )

                    mod_x_batch = get_batch_mod_feature(df_group.loc[i:batch_end,:], nAA)
                    mod_x = torch.Tensor(mod_x_batch)

                    CCSs = torch.Tensor(df_group.loc[i:batch_end,'CCS'].values)

                    charges = torch.Tensor(
                        df_group.loc[i:batch_end, 'charge'].values
                    ).unsqueeze(1)*self.charge_factor

                    cost = self._train_one_batch(
                        CCSs,
                        aa_indices, mod_x, charges
                    )
                    batch_cost.append(cost.item())
                if verbose_each_epoch:
                    batch_tqdm.set_description(
                        f'Epoch={epoch+1}, nAA={nAA}, Batch={len(batch_cost)}, Loss={cost.item():.4f}'
                    )
            if verbose: print(f'[MS/MS training] Epoch={epoch+1}, Mean Loss={np.mean(batch_cost)}')

    def predict(self, precursor_df, batch_size=1024, verbose=False):
        self.model.eval()

        precursor_df['predict_CCS'] = 0

        _grouped = precursor_df.groupby('nAA')
        df_list = []
        if verbose:
            batch_tqdm = tqdm(_grouped)
        else:
            batch_tqdm = _grouped
        for nAA, df_group in batch_tqdm:
            df_group = df_group.reset_index(drop=True)
            for i in range(0, len(df_group), batch_size):
                batch_end = i+batch_size-1 # DataFrame.loc[start:end] inlcudes the end

                mod_x_batch = get_batch_mod_feature(df_group.loc[i:batch_end,:], nAA)

                aa_indices = torch.LongTensor(parse_aa_indices(
                    df_group.loc[i:batch_end, 'sequence'].values.astype('U')
                ))
                mod_x = torch.Tensor(mod_x_batch)

                charges = torch.Tensor(
                    df_group.loc[i:batch_end, 'charge'].values
                ).unsqueeze(1)*self.charge_factor

                predicts = self.model(
                    *[fea.to(self.device) for fea in
                    [aa_indices, mod_x, charges]
                ]).cpu().detach().numpy()

                predicts[predicts<0] = 0

                df_group.loc[i:batch_end, 'predict_CCS'] = predicts
            df_list.append(df_group)

        return pd.concat(df_list).reset_index(drop=True)

In [ ]:
model = AlphaCCSModel()
model.device = torch.device('cpu')
model.model.to(model.device)
model.model(torch.LongTensor([[1,2,3,4,5,6]]), torch.tensor([[[0.0]*8]*6]), torch.tensor([[1]]))

tensor([0.0019], grad_fn=<SqueezeBackward1>)

In [ ]:
model.model

EncDecModelCCS(
  (dropout): Dropout(p=0.2, inplace=False)
  (encoder): SeqEncoder(
    (dropout): Dropout(p=0, inplace=False)
    (input_cnn): SeqCNN(
      (cnn_short): Conv1d(36, 36, kernel_size=(3,), stride=(1,), padding=(1,))
      (cnn_medium): Conv1d(36, 36, kernel_size=(5,), stride=(1,), padding=(2,))
      (cnn_long): Conv1d(36, 36, kernel_size=(7,), stride=(1,), padding=(3,))
    )
    (hidden_nn): SeqLSTM(
      (rnn): LSTM(144, 128, num_layers=2, batch_first=True, bidirectional=True)
    )
    (attn_sum): SeqAttentionSum(
      (attn): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=False)
        (1): Softmax(dim=1)
      )
    )
  )
  (decoder): LinearDecoder(
    (nn): Sequential(
      (0): Linear(in_features=257, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
      (2): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)